In [ ]:
import pandas as pd
import sklearn.feature_selection
import sklearn.metrics
import sklearn.model_selection
import sklearn.linear_model
import sklearn.covariance
import sklearn.cross_decomposition
import sklearn.feature_extraction
import sklearn.svm
import sklearn.compose

import statsmodels.api as sm
# from statsmodels.sandbox.regression.gmm import IV2SLS
from linearmodels.iv import IV2SLS

In [ ]:
combo_df = pd.read_csv('combined.csv')


In [ ]:
# print(combo_df.dtypes)

In [ ]:
df = combo_df.__deepcopy__()

# print(combo_df.isnull().sum().sum())
# print(121964*32)
df = df[df['Price'] >= 0]
df

In [ ]:
df = df.dropna(axis = 'columns')
df

In [ ]:
df['Date'] = pd.to_datetime(df['Date']).astype('int64')
df.dtypes

In [ ]:
df = df[['Date', 'CSA', 'CBSA', 'Total','1 Unit', '2 Units', '3 and 4 Units', '5 Units or More', 'Num of Structures With 5 Units or More', 'Year', 'Month', 'filename', 'Price']]
cols = df.columns.to_list()
cols.remove('filename')
scaler = sklearn.preprocessing.MinMaxScaler()
temp_df= df[['Date', 'CSA', 'CBSA', 'Total','1 Unit', '2 Units', '3 and 4 Units', '5 Units or More', 'Num of Structures With 5 Units or More', 'Year', 'Month', 'Price']]
# scaler.fit(temp_df)
temp_df = pd.DataFrame(scaler.fit_transform(temp_df))

temp_df.columns = cols
temp_df['filename'] = df['filename']

df = temp_df
#
df

In [ ]:
print(combo_df.columns)

In [ ]:
df['Single Unit ratio'] = df['1 Unit']/df['Total']

In [ ]:
df['Single Unit ratio'].max()

In [ ]:
df['Single Unit ratio'].min()

In [ ]:
default_corr = df.corr(method = 'spearman')

In [ ]:
print(default_corr['Price'])

In [ ]:
Pearson_corr = df.corr(method = 'pearson')
Pearson_corr

In [ ]:
print(Pearson_corr['Price'])

In [ ]:
kendall_corr = df.corr(method = 'kendall')
kendall_corr

In [ ]:
print(kendall_corr['Price'])

In [ ]:
linear_reg = sklearn.linear_model.LinearRegression()

ohe = sklearn.preprocessing.OneHotEncoder(sparse = False, handle_unknown='ignore')


In [ ]:
df = df.dropna(axis = 'rows')
print(df.isnull().sum().sum())



In [ ]:
X = df.drop(['Price'], axis = 'columns')
print(X.sample(5))
# print(X.head())
y = df[['Price']]
col_transform = sklearn.compose.make_column_transformer((ohe, ['filename']))

#
# X = pd.DataFrame(scaled, columns = X.columns)

col_transform.fit_transform(X)

print(col_transform)


In [ ]:
print(X)
print(y)

In [ ]:

endog = df.columns.to_list()
endog.remove('Price')
endog.remove('filename')

x_const = sm.add_constant(df[endog])
first_model_results = sm.OLS(y, x_const, missing= 'Drop').fit()
first_model_results.summary()


In [ ]:




# endog = df.columns.to_list()
# endog.remove('Price')
# endog.remove('filename')
print(endog)
iv2sls_model = IV2SLS(df['Price'], df[endog], None, None ).fit(cov_type="unadjusted")
print(iv2sls_model)

In [ ]:


pipe = sklearn.pipeline.make_pipeline(col_transform, linear_reg)

In [ ]:
sklearn.metrics.get_scorer_names()

In [ ]:
for scoring in sklearn.metrics.get_scorer_names():
    print(scoring)
    try:
        sklearn.model_selection.cross_val_score(pipe, X, y, cv=5, scoring = scorer).mean()
    except: continue

In [ ]:
# ind_vars = ['CSA', 'CBSA', 'Total', '1 Unit','2 Units', '3 and 4 Units', '5 Units or More',
#             'Num of Structures With 5 Units or More', 'Year', 'Month', 'Population 1', 'Population 5',
#             'Population Diff','Population', 'Pop Growth 1 Year', 'Pop -1 Years','Pop Percent 1 Year',
#             'Pop Growth 5 Year',
#             'Pop -5 Years','Pop Percent 5 Year', 'Single Unit ratio']
# dep_vars = ['Price']
#
# cols = ['CSA', 'CBSA', 'Total', '1 Unit','2 Units', '3 and 4 Units', '5 Units or More',
#         'Num of Structures With 5 Units or More', 'Year', 'Month', 'Population 1', 'Population 5',
#         'Population Diff','Population', 'Pop Growth 1 Year', 'Pop -1 Years','Pop Percent 1 Year',
#         'Pop Growth 5 Year','Pop -5 Years','Pop Percent 5 Year', 'Single Unit ratio', 'Price']
#
# # print(cols)
# no_null_df = combo_df[cols].dropna()
# # print(no_null_df.head())
# x = no_null_df[ind_vars]
# y = no_null_df[dep_vars]
# #
# print(x.head())
#
# x_const = sm.add_constant(x)
#
# first_model_results = sm.OLS(y, x_const).fit_regularized()
# first_model_results.summary()